## Analyzing contract by month

In [1]:
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import Scatter, Figure, Layout, Histogram, Heatmap
from plotly import tools
init_notebook_mode(connected=True)

from utils.plot_utils import plot, plot_subplot, data_from_table, engine, gen_datetime_col
# Grab our data again. 
df_query = data_from_table('cl_data', 'index')
df_query['datetime'] = gen_datetime_col(df_query, 'year', 'month', 'day')
df_query.head()

,open,high,low,close,volume,openint,contract_name,year,month,day,contract_symbol,contract_year,contract_month,datetime
index,,,,,,,,,,,,,,
0,29.01,29.56,29.01,29.40,949,470,CL1983-06,1983,3,30,CL,1983,6,1983-03-30
1,29.40,29.60,29.25,29.29,521,523,CL1983-06,1983,3,31,CL,1983,6,1983-03-31
2,29.30,29.70,29.29,29.44,156,583,CL1983-06,1983,4,4,CL,1983,6,1983-04-04
3,29.50,29.80,29.50,29.71,175,623,CL1983-06,1983,4,5,CL,1983,6,1983-04-05
4,29.90,29.92,29.65,29.90,392,640,CL1983-06,1983,4,6,CL,1983,6,1983-04-06


In [2]:
df_yearly_contract_month = df_query.groupby(('contract_year', 'contract_month')).mean().reset_index()
df = df_yearly_contract_month

# x entries are values going to be in the format YYYY-MM so plotly knows how to read them
f = lambda x: '{}-{:02d}'.format(x[0], x[1])
x = df[['contract_year', 'contract_month']].apply(f, axis=1)
title = 'Average value of contracts over its life'
plot([Scatter(x=x, y=df['open'], mode='markers')], title, interactive=False)

Woah! There are some wierd things going on here. There are two trendlines. Looking closer we can see that these are months that follow different trendlines. June and December in particular stand out:

In [3]:
df['open'].max()

92.83639892904948

In [4]:
# To Avoid stairstepping, let's plot all the contracts by month:
df_yearly_contract_month = df_query.groupby(('contract_year', 'contract_month')).mean().reset_index()
df = df_yearly_contract_month
df['contract_date'] = df[['contract_year', 'contract_month']].apply(lambda x: '{}-{:02d}'.format(x[0], x[1]), axis=1)
title = 'Average value of contracts over its life'

months = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
data = []
for m in range(1, 13):
    d = df[df['contract_month'] == m]
    data.append(Scatter(x=d['contract_date'], y=d['open'], name='{} Contract'.format(months[m])))

plot(data, title=title)